In [1]:
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate


In [2]:
llm = CTransformers(
            model='llama-2-7b.ggmlv3.q8_0.bin',
            model_type='llama',
            config={
                "max_new_tokens":512,
                "temperature":0.3,
                "context_length":2048
            }
        )

d:\Desktop\AI_LAB\ISRO\automated_test_case_generation_Copy\.isro\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
example = """{"variables" : ['a', 'b',....],
"equations" : [{'equation' : 'b = 2*d', 'condition': None},
                {'equation':  'a = b+c', 'condition': 'b>0'},
                {'equation':  'a = b+c', 'condition': 'mode == "enabled" and speed > 120'}.... {}]
}"""
output_condition = "Start your response with the json open brace '{' and end the response with '}'"

In [4]:
EQUATION_SYS_PROMPT_TEMPLATE = """
You are a python equation generator.
You will be given a functional requirement document of a satellite.
The document will contain conditions and equations of technicalities related to satellites in natural language.
Your task is to analyse the given document.
- Identify ALL the variables present and ALL the equations present in the document.
- For the equations, all the conditions and equation should be a python executable command or line which can be executed by eval.
- Give equations in the order of their executions. Use None instead of null. Identify ALL the variables present.
- You will be given example input document and example output JSON.
You have to return the output in the following JSON format.
```json
{example}
```
Example input document: {input_frd} 
Example output json: {output_frd}
{output_condition}
"""

USER_PROMPT = """
Input Document: {document}.
"""

EQUATION_PROMPT_TEMPLATE = f"""
<s>[INST]\n<<SYS>>\n{EQUATION_SYS_PROMPT_TEMPLATE}\n<</SYS>>\n\n{USER_PROMPT}[/INST]\nOutput Json: 
"""

In [5]:
prompt = PromptTemplate(
    input_variables=["input_frd","output_frd","document","example","output_condition"], template=EQUATION_PROMPT_TEMPLATE
)

In [10]:
FRD = '''
Auto_Drift_Estimate - ["enabled", "not_enabled"]
a0 - [1, 2]
a1 - [2, 2]
a2 - [3, 5]
a3 - [0, 2]
a4 - [0, 1]
a5 - [1, 2]
a6 - [0, 2]
actual_frequency - [7000, 7010]
present_temperature - [-10, 50]
RANGE_END_TAG

If Auto_Drift_Estimate Enabled
If -10 < present_temperature < 55 then
delfbyf0 = a0 + a1 * (present_temperature - temp0) + a2 * (present_temperature - temp0)^2 + a3 * (present_temperature - temp0)^3 + a4 * (present_temperature - temp0)^4 + a5 * (present_temperature - temp0)^5 + a6 * (present_temperature - temp0)^6
Where
f0 = (1/0.000125)
theoretical_frequency = f0
deltaf = actual_frequency - theoretical_frequency
temp0 = 25 deg
The above logic shall be computed every 7 sec as the temperature reading shall happen every 7 sec.
deltaf = f0 * delfbyf0
new_time_period = 1/(f0 + deltaf)
delta_time_period = new_time_period - t0
'''


input_frd = """
Auto_drift - [0, 1, 0.5]
operation - ["fight", "no"]
curvature - [100, 102, 1]
_RANGE_END_TAG_

beta = 100
temp0 = 40 deg
If Auto_drift is 0,
then beam = curvature + 5 + temp
else,
beam = curvature * 5
"""

output_frd = """
{
"variables": ['beta', 'temp0', 'Auto_drift', 'curvature', 'beam'],
"equations": [
{'equation': 'beta = 100', 'condition': None}, 
{'equation': 'temp0 = 40', 'condition': None}, 
{'equation': 'beam = curvature + 5 + temp', 'condition': 'Auto_drift == 0 and operation == "fight"'}, 
{'equation':  'beam = curvature * 5', 'condition': 'Auto_drift != 0'} 
]
}
"""

In [7]:
op = prompt.format_prompt(input_frd=input_frd,
                     output_frd=output_frd,
                     document=FRD,
                     example=example,
                     output_condition=output_condition)

In [9]:
llm_chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
completion = llm_chain.predict(input_frd=input_frd,
                     output_frd=output_frd,
                     document=FRD,
                     example=example,
                     output_condition=output_condition)

In [ ]:
print(completion)

In [ ]:
print(completion.text)

In [ ]:
print("thank you good night")